In [42]:
import geopandas as gpd
import pandas as pd
import sys
import numpy as np
import rasterio as rs
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
#from rasterstats import zonal_stats

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
def reproject_to_meters(input_raster, output_raster, target_crs='EPSG:3857'):
    
    with rs.open(input_raster) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rs.open(output_raster, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rs.band(src, i),
                    destination=rs.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest
                )

In [22]:
prj_districts = gpd.read_file('../../data/epa_districts/Project_Districts.shp')
input_raster = 'final_2024-10-29.tif'
repj_raster = 'reprojected_2024-10-29.tif'

# reproject inputs from degrees to meters for area assessment calcs
dst_crs = 'EPSG:3857'
reproject_to_meters(input_raster, repj_raster, dst_crs)
prj_districts = prj_districts.to_crs(dst_crs)

In [69]:
zonal_stats = []
with rs.open(repj_raster) as src:
    for _, district in prj_districts.iterrows():
        out_image, out_transform = mask(src, [district.geometry], crop=True)
        district_mask = out_image[0] 
        unique, counts = np.unique(district_mask, return_counts=True)
        land_use_stats = dict(zip(unique, counts))

        # calculate pixel size to convert to ha
        pixel_width = src.transform[0]  # X resolution, 10.14
        pixel_height = -src.transform[4]  # Y resolution
        px_size = pixel_width * pixel_height
        land_use_stats = {k: v * (px_size/10000) for k, v in land_use_stats.items()}
        
        land_use_stats['district'] = district.ADM2_EN
        zonal_stats.append(land_use_stats)

df = pd.DataFrame(zonal_stats)
df = df.round(2).rename(columns= {
                        0: "no_tree",
                        1: "monoculture",
                        2: "agroforestry",
                        3: "natural",
                        255: "no_data"
                    })
df.to_csv('../../data/area_assessment.csv', index=False)

In [59]:
df

,no_tree,monoculture,agroforestry,natural,no_data,district
0,5957.77,1072.42,53783.13,16628.07,57926.96,Adansi South
1,4611.13,655.26,97541.32,14791.46,135822.38,Asante Akim South
2,3359.99,729.69,66770.66,2964.08,51943.00,Assin North
3,11460.29,5126.34,159426.26,16005.69,168338.32,Atwima Mponua
4,92008.14,5.06,16625.81,2381.25,90072.34,Bawku West
5,3537.51,743.41,41715.90,11938.37,98417.49,Bosome Freho
6,99763.55,0.92,25806.06,3711.35,195523.07,Builsa South
7,114226.55,0.38,17874.65,19311.11,142670.70,Daffiama Bussie Issa
8,1375.07,549.70,12402.00,3137.66,40548.41,Juaben Municipal
9,68388.22,12.67,20163.26,2528.68,146231.46,Kasena Nankana West


In [67]:
#calculate total area of raster
src = rs.open(repj_raster)
data = src.read(1)
valid_px = np.sum(data != src.nodata)
pixel_width = src.transform[0]  # X resolution
pixel_height = -src.transform[4]  # Y resolution
px_size = pixel_width * pixel_height
total_area_in_m2 = valid_px * (px_size)
total_area_in_ha = total_area_in_m2 / 10000

# sum all area assessments
# drops nodata column
district_total_area = df.iloc[:, :-2].sum().sum()

round(total_area_in_ha), round(district_total_area)

(5247876, 5247333)

In [70]:
df['ag_area'] = df.agroforestry + df.monoculture
df['nat_area'] = df.natural
df_pubfigure = df[['district', 'ag_area', 'nat_area']]
df_pubfigure

,district,ag_area,nat_area
0,Adansi South,54855.55,16628.07
1,Asante Akim South,98196.58,14791.46
2,Assin North,67500.35,2964.08
3,Atwima Mponua,164552.60,16005.69
4,Bawku West,16630.87,2381.25
5,Bosome Freho,42459.31,11938.37
6,Builsa South,25806.98,3711.35
7,Daffiama Bussie Issa,17875.03,19311.11
8,Juaben Municipal,12951.70,3137.66
9,Kasena Nankana West,20175.93,2528.68


In [72]:
df_pubfigure.to_csv('../../data/area_assessment_figure.csv')

In [51]:
# ## Double check w/ Rasterstats Approach
# lu_cats = {
#     0: "no_tree",
#     1: "monoculture",
#     2: "agroforestry",
#     3: "natural",
#     255: "no_data"
# }
# stats = zonal_stats('../../data/epa_districts/Project_Districts.shp',
#            'final_2024-10-29.tif',
#             categorical=True, 
#             category_map=lu_cats,
#             stats="count min mean max median")